# Anwendung: Laststeuerung in der Stahlproduktion

Die Roheisen- und Stahlerzeugung ist derzeit für etwa 6,4\% der $CO_2$-Emissionen in Deutschland verantwortlich. Eine Möglichkeit, diese Emissionen zu reduzieren, ist der Einsatz von sogenannten elektrischen Lichtbogenöfen, die mit Strom aus erneuerbaren Energien betrieben werden. In diesen wird Eisen mittels elektrischer Energie eingeschmolzen und zu Stahl weiterverarbeitet. Da erneuerbare Energien in der Erzeugung sehr volatil sind und daher im Preis je nach Wetter und Tageszeit stark schwanken, ist ein Ansatz die sog. Laststeuerung, d.h. Verschiebung von energieintensiven Prozessen in Zeiträume, bei denen viel Strom aus erneuerbaren Energien vorhanden und somit günstig ist.

## Aufgabenstellung

In diesem Abschnitt soll anhand eines (fiktiven und vereinfachten) Modells untersucht werden, wie mittels linearer Optimierung Produktionsprozesse zeitlich verschoben und so insgesamt günstiger produziert werden kann. Gegeben sind folgende Daten:

In [56]:
import pandas as pd

df = pd.read_csv("Daten/production_data.csv")
df.head()

,Zeit,Produktionskosten [EUR/t],Bestellungen [t]
0,1,240,22
1,2,540,27
2,3,660,25
3,4,900,8
4,5,600,0


Die Daten sind wie folgt zu verstehen: ``Produktionskosten [EUR/t]`` sind die gesamten variablen Produktionskosten (Energie, Rohmaterial, Personal) für eine Tonne Stahl während Zeitperiode $T$ (eine Zeitperiode entspricht hier 6 Stunden). Die Produktionskosten spiegeln im wesentlichen den Strompreis wider, der je nach Tageszeit und Wetterlage stark schwankt. Wir gehen davon aus, dass das Stahlwerk einen Vertrag mit dem Energieversorger hat, in dem der Strompreis zwar variabel ist, aber für eine Woche im Voraus bekannt ist. ``Bestellungen [t]`` beschreibt die Menge an Stahl, der in dieser Periode (an Kunden und/oder nachgelagerte Prozesse) bereitgestellt werden muss (durch Produktion und/oder zuvor eingelagertem Stahl).

Pro Zeitperiode können maximal $50t$ Stahl produziert werden. Weiterhin existiert ein Lager, in welches maximal 20t Stahl eingelagert werden können. Zum Zeitpunkt $T=0$ befinden sich in dem Lager $15t$ Stahl.

Neben den reinen Produktionskosten fallen während der Produktion weitere Kosten an: 

- Sobald in einer Periode Stahl produziert wird, fallen Rüstkosten von 2000 EUR für die Inbetriebnahme der Maschinen an. Falls in einer Zeitperiode nichts produziert wird, fallen auch keine Rüstkosten an. 
- Pro eingelagerter Tonne Stahl fallen 20 EUR Lagerhaltungskosten je Zeitperiode an.
- Pro Tonne Stahl werden 1000 EUR Umsatz erlöst (unabhängig von der Zeitperiode).

Für jede Zeitperiode soll entschieden werden, wieviel Tonnen Stahl produziert werden und wieviel ein- bzw. ausgelagert werden. Die Bestellmenge in jeder Zeitperiode muss exakt gedeckt werden. Als Zielfunktion soll der Gesamtgewinn, also der Umsatz abzüglich aller Kosten, summiert über alle Zeitperioden, maximiert werden. 

## Problemdaten und Indexmengen
Wir formalisieren alle relevanten im Text angegebenen Größen:
- Zeithorizont: $H=\{1,2,\dots,44\}$ 
- Umsatz pro Tonne Stahl in Euro: $U=1000$
- Maximale Produktionskapazität pro Zeitperiode in Tonnen: $C=50$ 
- Maximale Lagerkapazität in Tonnen $L=20$
- Lagerhaltungskosten pro Zeitperiode in Euro: $h=20$ 
- Rüstkosten, falls produziert wird in Euro: $R=2000$ EUR
- Variable Produktionskosten zum Zeitpunkt $t$ in Euro pro Tonne: $p_t$
- Bestellungen / Bedarf in Zeitperiode $t$ in Tonnen: $d_t$
- Den Lagerstand zu Beginn in Tonnen: $s_0=15$

In [57]:
H = list(range(1,len(df)+1))
U = 1000
C = 40
L = 20
h = 20
R = 2000
p = df["Produktionskosten [EUR/t]"]
p.index = df["Zeit"]
d = df["Bestellungen [t]"]
d.index = df["Zeit"]
s0 = 15

## Optimierungsvariablen

In jeder Zeitperiode sollen folgende Größen optimal bestimmt werden:
- Die Produktionsmenge $k_t\in\R$
- Die Menge $s_t^+\in\R$, die eingelagert wird
- Die Menge $s_t^-\in\R$, die ausgelagert wird
- Der Lagerbestand $s_t\in\R$
- Die Entscheidung, ob produziert werden soll:

    $$
        z_t=\left\{\begin{array}{rl}
        1 & \text{, in Zeitperiode }t\text{ wird produziert.} \\
        0 & \text{, sonst.}
        \end{array}\right.
    $$

Wir erstellen ein Gurobi Modell und legen die Variablen an.

In [58]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

m = gp.Model("Stahlproduktion")

# Wie viel soll in Periode t produziert werden?
k = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0)

# Ein- und Ausspeicherung während Periode t
s_in = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0)
s_out = m.addVars(H, vtype=GRB.CONTINUOUS, lb=0)

# Lagerstand während Periode t
s = m.addVars(np.hstack([0,H]), vtype=GRB.CONTINUOUS, lb=0)

# Soll in Periode t produziert werden?
z = m.addVars(H, vtype=GRB.BINARY)

## Zielfunktion

Es soll der Profit maximiert werden. Dieser berechnet sich pro Zeitperiode durch den Umsatz abzüglich Rüstkosten, variable Produktionskosten und Lagerhaltungskosten. Mathematisch:

$$
\max\quad \sum_{t\in H} d_tU - z_tR - p_tk_t - hs_t
$$


In [59]:
m.setObjective(gp.quicksum( U*d[t] - R*z[t] - h*s[t] - p[t]*k[t] for t in H), GRB.MAXIMIZE)

## Nebenbedingungen
Wir formulieren folgende Nebenbedingungen:

- Der Lagerstand zu Beginn ist festgelegt:

    $$
        s_0 = 15
    $$

- Lagerbilanz: Lagerstand ist Einlagerung minus Auslagerung plus vorheriger Lagerbestand:

    $$
        s_t = s_{t-1} - s_t^- + s_t^+,\quad t\in H
    $$

- Der Bedarf muss in jeder Zeitperiode erfüllt sein (durch Neuproduktion und Auslagerun):

    $$
        d_t = k_t + s_t^- - s_t^+,\quad t\in H
    $$

- Die Lagerkapazität muss in jeder Zeitperiode berücksichtigt werden:
    
    $$
        s_t \leq L,\quad t\in H
    $$

- Die Produktionskapazität muss berücksichtigt werden. Nur wenn nichts produziert wird ($k_t=0$), kann $z_t=0$ werden.
    
    $$
        k_t\leq Cz_t,\quad t\in H
    $$

In [60]:
# Constraint: Lagerbestand zu Beginn
m.addConstr(s[0] == 15)

# Constraints: Lagerbilanz
m.addConstrs((s[t] == s[t-1] - s_out[t] + s_in[t] for t in H))

# Constraints: Bedarf muss erfüllt sein
m.addConstrs((k[t] + s_out[t] - s_in[t] == d[t] for t in H))

# Constraints: Lagerkapazität
m.addConstrs((s[t] <= L for t in H))

# Constraints: Produktionskapazität
m.addConstrs((k[t] <= C*z[t] for t in H))

m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 177 rows, 221 columns and 441 nonzeros
Model fingerprint: 0xcdff404a
Variable types: 177 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 86560.000000
Presolve removed 131 rows and 154 columns
Presolve time: 0.00s
Presolved: 46 rows, 67 columns, 112 nonzeros
Found heuristic solution: objective 134040.00000
Variable types: 47 continuous, 20 integer (20 binary)

Root relaxation: objective 2.102830e+05, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

Wir berechnen einige Kennzahlen der Lösung und speichern die Zeitreihen für Lagerbestand und Produktionsmenge im Dataframe mit den Problemdaten zur späteren Visualisierung.

In [61]:
df["Lagerbestand [t]"] = [s[t].x for t in H] 
df["Produktionsmenge [t]"] = [k[t].x for t in H] 
df["Produktionskapazität [t]"] = C
df["Lagerkapazität [t]"] = L

print(f"Gesamtgewinn laut Modell: {sum( U*d[t] - R*z[t] - h*s[t] - p[t]*k[t] for t in H).getValue()}") 
print(f"Lagerhaltungskosten gesamt: {sum(h*s[t] for t in H).getValue()}") 
print(f"Rüstkosten gesamt: {sum( R*z[t] for t in H).getValue()}") 
print(f"Produktionskosten gesamt: {sum(p[t]*k[t] for t in H).getValue()}") 

Gesamtgewinn laut Modell: 206900.0
Lagerhaltungskosten gesamt: 8220.0
Rüstkosten gesamt: 56000.0
Produktionskosten gesamt: 397880.0


Wir visualisieren die Ergebnisse als Zeitreihe (die Lagerkapazität und Produktionskapazität werden als waagrechte Linie angezeigt und dienen der Orientierung). Wir sehen eine Art Zickzack-Muster bei der Produktionsmenge und dem Lagerbestand. Hier versucht die Lösung, kurzfristige Schwankungen beim Produktionspreis auszunutzen und auf Vorrat zu produzieren. Zusätzlich ergibt sich dadurch die Möglichkeit, Rüstkosten einzusparen.

In [62]:
import plotly.express as px

px.line(data_frame=df, x="Zeit", y=["Produktionsmenge [t]", 
                                    "Produktionskapazität [t]", 
                                    "Lagerbestand [t]",
                                    "Lagerkapazität [t]",
                                    "Bestellungen [t]"])

In [63]:
px.line(data_frame=df, x="Zeit", y="Produktionskosten [EUR/t]")